In [4]:
import pandas as pd
import numpy as np
from lib import climb_jor, climber_class, grades_class

# Route recommender

In this notebook will perform the route recommending based on the user features

### We have 3 types of data:
- Independent features (clustered)
- How good the route is (the higher the better)
- `Grade, Location, heigh (defined by user)` => <b>In this notebook we will work on this one

### Create the user from climber class

In [5]:
climber_1 = climber_class.climber()
climber_2 = climber_class.climber(grade = '7a+')
climber_2.set_attributes(cluster=[-3,4,11,-1,30,-29,3,2,8])
display(climber_2.get_data())

Climber class initialized
Climber class initialized


,climber_id,name,ascents,grade_fra,grade,grade_range,country,crag,sector,height
0,3,climber3,0,7a+,51,2,esp,montserrat,agulla del senglar,170


### Retrieve routes table

In [6]:
routes = pd.read_csv('../data/routes_rated.csv',low_memory=False, index_col=0)
routes.head(2)

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot
0,0,and,montserrat,prohibitivo,diagonal de la x,-1,49.25,0,-0.045211
1,1,and,montserrat,prohibitivo,mehir,-1,49.00,0,0.116464


### Cleaning the table

Objective
- We will recommend 3 routes per each zone (country, crag, sector)

The logical tree will be
- Make 3 different filters for: sector, crag, country
- Filter tall recommended (if height<-1 and tall or if height>1 and small) tall is mean+5 (>180), short is mean-5 (<170)
- Filter by grade range, default +-2 (e.g. for a 7b grade => range[7a+,7b+]), if not routes ask to make the range bigger
- Order by overall rating
- Filter preferred clusters (If not then show other results)

#### a) make zone tables

In [7]:
climber_2.get_data()

,climber_id,name,ascents,grade_fra,grade,grade_range,country,crag,sector,height
0,3,climber3,0,7a+,51,2,esp,montserrat,agulla del senglar,170


In [8]:
routes_country = routes[routes.country == climber_2.get_data().country[0]]
routes_crag = routes[routes.crag == climber_2.get_data().crag[0]]
routes_sector = routes[routes.sector == climber_2.get_data().sector[0]]
print("Country routes: ", routes_country.shape[0])
print("Crag routes: ", routes_crag.shape[0])
print("Sector routes: ", routes_sector.shape[0])

Country routes:  15293
Crag routes:  797
Sector routes:  17


#### b) Filter tall recommend

In [9]:
climber_1.set_attributes(height=165)
climber_2.set_attributes(height=185)

Test with short

In [10]:
routes_country_tall = climb_jor.filter_tall(routes_df = routes_country, climber_usr = climber_1, show = True)
routes_crag_tall = climb_jor.filter_tall(routes_df = routes_crag, climber_usr = climber_1, show = True)
routes_sector_tall = climb_jor.filter_tall(routes_df = routes_sector, climber_usr = climber_1, show = True)

df routes:  12521
df routes:  646
df routes:  16


Test with tall

In [11]:
routes_country_tall = climb_jor.filter_tall(routes_df = routes_country, climber_usr = climber_2, show = True)
routes_crag_tall = climb_jor.filter_tall(routes_df = routes_crag, climber_usr = climber_2, show = True)
routes_sector_tall = climb_jor.filter_tall(routes_df = routes_sector, climber_usr = climber_2, show = True)

df routes:  12801
df routes:  699
df routes:  14


#### c) Filter grade range

In [12]:
routes_country_grade = climb_jor.filter_grade(routes_df = routes_country_tall, climber_usr = climber_2, show = True)
routes_crag_grade = climb_jor.filter_grade(routes_df = routes_crag_tall, climber_usr = climber_2, show = True)
routes_sector_grade = climb_jor.filter_grade(routes_df = routes_sector_tall, climber_usr = climber_2, show = True)

df routes:  1754
df routes:  113
df routes:  1


#### d) Order by overall rating

In [13]:
routes_country_rating = routes_country_grade.sort_values(by = 'rating_tot',ascending = False)
routes_crag_rating = routes_crag_grade.sort_values(by = 'rating_tot',ascending = False)
routes_sector_rating = routes_sector_grade.sort_values(by = 'rating_tot',ascending = False)

#### d) Filter clustering

In [14]:
routes_country_grade = climb_jor.filter_cluster(routes_df = routes_country_rating, climber_usr = climber_2, show = True)
routes_crag_grade = climb_jor.filter_cluster(routes_df = routes_crag_rating, climber_usr = climber_2, show = True)
routes_sector_grade = climb_jor.filter_cluster(routes_df = routes_sector_rating, climber_usr = climber_2, show = True)

df routes:  1754
df routes:  113
df routes:  1


In [15]:
climber_2.get_cluster_order()

[4, 2, 8, 1]

#### f) 3 per zone

In [16]:
display(routes_country_grade.head(3))
display(routes_crag_grade.head(3))
display(routes_sector_grade.head(3))

,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
21880,21880,esp,rodellar,palomera,super remedios,51.000000,2,0.480442,0
20436,20436,esp,pedriza,peseta,libro del fonambulista l,49.333333,2,0.470268,0
20704,20704,esp,penahorada,pena alta,villachula,50.500000,2,0.456653,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19015,19015,esp,montserrat,vermell,omaticaia,51.0,2,-0.105057,0
18593,18593,esp,montserrat,collbato,penya del cargol,51.0,1,0.499660,0
18770,18770,esp,montserrat,prohibitivo,ladvocat del diable,51.5,1,0.379970,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus


## Create the function

In [17]:
def route_recommender(climber, show = False):
    '''
    This function returns the route recommendations for the user "climber"
    inputs:
    - climber: the user from the climber class
    - show: True for showing the log
    
    output:
    df with the recommendations       
    '''
    
    
    def recommender_filter(df, climber, show = True):
        routes = df.copy()
         # Climber Filters    
        routes_tall = climb_jor.filter_tall(routes_df = routes, climber_usr = climber, show = show)
        routes_grade = climb_jor.filter_grade(routes_df = routes_tall, climber_usr = climber, show = show)

        # Rating Order
        routes_rating = routes_grade.sort_values(by = 'rating_tot',ascending = False)

        # Clustering Order
        routes_grade = climb_jor.filter_cluster(routes_df = routes_rating, climber_usr = climber, show = show)

        return routes_grade
    
    
    routes = pd.read_csv('../data/routes_rated.csv',low_memory=False, index_col=0)
       
    # Region
    routes_country = routes[routes.country == climber.get_data().country.values[0]]
    routes_crag = routes[routes.crag == climber.get_data().crag.values[0]]
    routes_sector = routes[routes.sector == climber.get_data().sector.values[0]]
    
    # Recommen
    routes_country_rec = recommender_filter(routes_country, climber, show)
    routes_crag_rec = recommender_filter(routes_crag, climber, show)
    routes_sector_rec = recommender_filter(routes_sector, climber, show)
    
    display(routes_country_rec.head(3))
    display(routes_crag_rec.head(3))
    display(routes_sector_rec.head(3))
       

In [18]:
route_recommender(climber_2)

,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
21880,21880,esp,rodellar,palomera,super remedios,51.000000,2,0.480442,0
20436,20436,esp,pedriza,peseta,libro del fonambulista l,49.333333,2,0.470268,0
20704,20704,esp,penahorada,pena alta,villachula,50.500000,2,0.456653,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19015,19015,esp,montserrat,vermell,omaticaia,51.0,2,-0.105057,0
18593,18593,esp,montserrat,collbato,penya del cargol,51.0,1,0.499660,0
18770,18770,esp,montserrat,prohibitivo,ladvocat del diable,51.5,1,0.379970,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus


In [19]:
country = 'esp'
crag = 'montserrat'
sector = 'el vermell'

climber_3 = climber_class.climber(name = 'Jordi',
                                  grade = '7b+',
                                  grade_range = 4,
                                  cluster = [0,0,2,0,5,6,3,4,0],
                                  location = [country, crag, sector])

Climber class initialized


In [20]:
route_recommender(climber_3)

,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
23589,23589,esp,siurana,salt de la reina mora,microcosmos,53.5,0,0.921312,0
10750,10750,esp,archidona,cueva grande,bujarros,53.5,0,0.882098,0
13662,13662,esp,collsuspina,parking,sumum,53.0,0,0.864996,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19088,19088,esp,montserrat,vermell,sin nombre,51.181818,0,0.603697,0
18606,18606,esp,montserrat,collbato,saltimbanqui,51.500000,0,0.439815,0
18556,18556,esp,montserrat,clot del boixar,senyor arboc,53.000000,0,0.399253,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus


In [21]:
montse = climb_jor.Crag_search(routes, 'montserrat')

In [22]:
montse.cluster.value_counts()

0    471
1    315
2      7
3      4
Name: cluster, dtype: int64

In [23]:
montse[montse.tall_recommend_sum < -2]

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot
18429,18429,esp,montserrat,can jorba,guakamayo,-3,53.600000,0,0.016613
18627,18627,esp,montserrat,colonia puig,joya de la corona,-3,63.571429,0,0.211878
18642,18642,esp,montserrat,cova de larcada,good feeling,-3,49.928571,1,0.072921
18737,18737,esp,montserrat,piula,marc attack,-3,52.928571,1,0.069647
18769,18769,esp,montserrat,prohibitivo,kill bill,-4,50.555556,1,-0.038214
18840,18840,esp,montserrat,vermell,aniversari,-3,40.243243,1,-0.021643
19078,19078,esp,montserrat,vermell,sessio digital,-3,54.515152,0,0.018349
19082,19082,esp,montserrat,vermell,shin shat,-3,50.821429,1,0.030656
19129,19129,esp,montserrat,vermell,via kpujo,-5,38.500000,0,-0.117026


In [24]:
a,b,c = climber_2.route_recommender(show = True)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/routes_rated.csv'

In [ ]:
cl = climber_class.climber()

a,b,c = cl.route_recommender(show = True)